In [1]:
import fetch_data
import nest_asyncio

lv1_area = fetch_data.fetch_subarea('360')

nest_asyncio.apply()
lv1_area_codes = [i['kodeWilayah'] for i in lv1_area if i['namaWilayah'] != 'LUAR NEGERI']
lv2_area = fetch_data.crawl_subareas(lv1_area_codes)
lv2_area_codes = [i['kodeWilayah'] for i in lv2_area]
lv3_area = fetch_data.crawl_subareas(lv2_area_codes)

level1= []
for prov in lv1_area:
    item = {}
    item['kodeProv'] = prov['kodeWilayah']
    item['kodeProvDagri'] = prov['kodeWilayahDagri']
    item['namaProv'] = prov['namaWilayahDagri'].strip()
    level1.append(item)

level2= []
for kabkot in lv2_area:
    item2 = {}
    item2['kodeKabKot'] = kabkot['kodeWilayah']
    item2['kodeKabKotDagri'] = kabkot['kodeWilayahDagri']
    item2['namaKabKot'] = kabkot['namaWilayahDagri'].strip()
    item2['kodeProv'] = kabkot['kodeIndukWilayah']
    level2.append(item2)

level3= []
for kec in lv3_area:
    item3 = {}
    item3['kodeKec'] = kec['kodeWilayah']
    item3['kodeKecDagri'] = kec['kodeWilayahDagri']
    item3['namaKec'] = kec['namaWilayahDagri'].strip()
    item3['kodeKabKot'] = kec['kodeIndukWilayah']
    level3.append(item3)

In [4]:
merged_test = []
for h in level3:
    for i in level2:
        for j in level1:
            if h['kodeKabKot'] == i['kodeKabKot'] and i['kodeProv'] == j['kodeProv']:
                merged_dict = h|i|j
            else:
                None
    merged_test.append(merged_dict)

In [11]:
import pandas as pd
jatim = [j for j in merged_test if j['namaProv'] == 'JAWA TIMUR']
pd.DataFrame(jatim).to_csv('kode_jatim.csv', index=False)

In [1]:
import csv
import fetch_data
import nest_asyncio
import logging
import sys

logstream_handler = logging.StreamHandler(stream=sys.stdout)
logging.basicConfig(handlers=[logstream_handler],
                    format='%(asctime)s.%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

nest_asyncio.apply()

with open('kode_jatim.csv', 'r') as f:
    wilayah = f.read().splitlines()

sel_wilayah = [i['kodeKec'] for i in csv.DictReader(wilayah)]

cuba = fetch_data.crawl_schlists(sel_wilayah)

In [ ]:
import httpx
import asyncio

import logging

import sys


from fetch_data import TARGET_BASE_URL

from httpx import ReadTimeout, RemoteProtocolError

logstream_handler = logging.StreamHandler(stream=sys.stdout)
logging.basicConfig(handlers=[logstream_handler],
                    format='%(asctime)s.%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

def paginator(items:list, items_per_page:int):
    pages = [items[item:item+items_per_page] for item in range(0, len(items), items_per_page)]
    return pages

async def _crawl(_area:str) -> list:
    async with httpx.AsyncClient(timeout=None) as client:
        # try:
            data_url = f'{TARGET_BASE_URL}satuan-pendidikan/download?kodeKecamatan={_area}&sortBy=bentuk_pendidikan&sortDir=asc&format=csv'
            fetch_raw = await client.get(data_url)
            resp = fetch_raw.content.decode('UTF-8')
            parse_data =  csv.DictReader(resp.splitlines())
            stacked = [i for i in parse_data]
            
            return stacked
        # except ReadTimeout:
        #     print(data_url)
        #     return exit(0)
        
        
async def job_aggregator(job_list):
    job_agg = await asyncio.gather(*job_list)
    return job_agg

sch_list = []
for subset in paginator(sel_wilayah, 50):
    joblist = [_crawl(area) for area in subset]
    sequences = asyncio.run(job_aggregator(joblist))
    sch_list.append(sequences)

In [4]:
unnest_data = [datum for array_datum in sch_list for datum in array_datum]

In [6]:
unnest_data2 = [datum for array_datum in unnest_data for datum in array_datum]

In [8]:
import pandas as pd
df = pd.DataFrame(unnest_data2)

In [ ]:
df